In [37]:
#dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import json



sys.path.insert(1,'/Users/Trav/Desktop/Coding/API_PACKAGE')
import api_keys



In [38]:
#importing the kaggle data
kaggle_spot_df = pd.read_csv("Resources/kaggle_spotify.csv")

kaggle_spot_df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.995,['Carl Woitschach'],0.708,158648,0.1950,0,6KbQ3uYMLKb5jDxLF7wYDD,0.563,10,0.1510,-12.428,1,Singende Bataillone 1. Teil,0,1928,0.0506,118.469,0.7790,1928
1,0.994,"['Robert Schumann', 'Vladimir Horowitz']",0.379,282133,0.0135,0,6KuQTIu1KoTTkLXKrwlLPV,0.901,8,0.0763,-28.454,1,"Fantasiestücke, Op. 111: Più tosto lento",0,1928,0.0462,83.972,0.0767,1928
2,0.604,['Seweryn Goszczyński'],0.749,104300,0.2200,0,6L63VW0PibdM1HDSBoqnoM,0.000,5,0.1190,-19.924,0,Chapter 1.18 - Zamek kaniowski,0,1928,0.9290,107.177,0.8800,1928
3,0.995,['Francisco Canaro'],0.781,180760,0.1300,0,6M94FkXd15sOAOQYRnWPN8,0.887,1,0.1110,-14.734,0,Bebamos Juntos - Instrumental (Remasterizado),0,1928-09-25,0.0926,108.003,0.7200,1928
4,0.990,"['Frédéric Chopin', 'Vladimir Horowitz']",0.210,687733,0.2040,0,6N6tiFZ9vLTSOIxkj8qKrd,0.908,11,0.0980,-16.829,1,"Polonaise-Fantaisie in A-Flat Major, Op. 61",1,1928,0.0424,62.149,0.0693,1928


In [39]:
#agg mean of spotify metrics by year
annual_metrics_df=kaggle_spot_df[['year','acousticness','danceability','energy','instrumentalness','liveness','loudness', 'popularity','speechiness','tempo','valence']].groupby('year').mean()

annual_metrics_df


,acousticness,danceability,energy,instrumentalness,liveness,loudness,popularity,speechiness,tempo,valence
year,,,,,,,,,,
1921,0.895823,0.425661,0.236784,0.322330,0.215814,-17.095438,0.351562,0.077258,100.397758,0.425495
1922,0.939236,0.480000,0.237026,0.440470,0.238647,-19.179958,0.138889,0.115419,101.376139,0.534056
1923,0.976329,0.568462,0.246936,0.401932,0.236656,-14.373882,5.727811,0.098619,112.456598,0.624788
1924,0.935575,0.548654,0.347033,0.583955,0.237875,-14.202304,0.603376,0.090210,120.653359,0.668574
1925,0.965422,0.571890,0.264373,0.408893,0.243094,-14.516707,2.707224,0.115457,115.671715,0.616430
...,...,...,...,...,...,...,...,...,...,...
2016,0.280290,0.599976,0.592877,0.074646,0.180198,-7.949913,61.371254,0.107298,119.070344,0.430769
2017,0.289916,0.612286,0.586739,0.098209,0.194218,-8.422697,64.861500,0.111752,116.840278,0.414465
2018,0.271941,0.664930,0.590591,0.035948,0.171781,-7.253666,67.276000,0.128140,122.004325,0.447141


In [40]:
#turns the metric names into a list
spot_metric_names=annual_metrics_df.columns.to_list()

In [41]:
## songs that are unique, popular, and stand out among other songs in a year
## conceptually, in this cell I want to look for popular songs in each year that broke the norm in regards the spotify metrics.
#look for outliers for each metric, we'll also want to compare those to popularity to see if any song's were unique (in regards to the metric) but also popular


# #creates a list of each year
# years_list=kaggle_spot_df['year'].unique().tolist()
# # print(type(years_list))
# years_list

#### the functions below find outlier songs within a given year for a specific spotify metric. The first finds all the outliers, the 'upper' finds the outliers on the top end, the 'lower' finds the ourliers on the bottom end


def outlier_songs(metric,year):
    # finds all the quartile related stats for each metric in a given year
    annual_q_stats_df=kaggle_spot_df.loc[kaggle_spot_df['year']==year,spot_metric_names].describe()
    #find outliers in the acousticness

    iqr=annual_q_stats_df[metric]['75%']-annual_q_stats_df[metric]['25%']
    iqr
    upperbound=annual_q_stats_df[metric]['75%']+(1.5*iqr)
    lowerbound=annual_q_stats_df[metric]['25%']-(1.5*iqr)

    # i could look at above average popularity or popularity above the 75 percentile
    avg_poularity=annual_q_stats_df['popularity']['mean']
    high_popularity=annual_q_stats_df['popularity']['75%']

    outliers_df=kaggle_spot_df.loc[(kaggle_spot_df['year']==year)&((kaggle_spot_df[metric]>upperbound)|(kaggle_spot_df[metric]<lowerbound)),:]
    outliers_df
    ### maybe i should look at popularity beyond an upper bound, instead of more than average popularity
    popular_outliers_df=outliers_df.loc[outliers_df['popularity']>avg_poularity]
    return popular_outliers_df


def upper_outlier_songs(metric,year):
    # finds all the quartile related stats for each metric in a given year
    annual_q_stats_df=kaggle_spot_df.loc[kaggle_spot_df['year']==year,spot_metric_names].describe()
    #find outliers in the acousticness

    iqr=annual_q_stats_df[metric]['75%']-annual_q_stats_df[metric]['25%']
    iqr
    upperbound=annual_q_stats_df[metric]['75%']+(1.5*iqr)
    lowerbound=annual_q_stats_df[metric]['25%']-(1.5*iqr)

    # i could look at above average popularity or popularity above the 75 percentile
    avg_poularity=annual_q_stats_df['popularity']['mean']
    high_popularity=annual_q_stats_df['popularity']['75%']


    outliers_df=kaggle_spot_df.loc[(kaggle_spot_df['year']==year)&((kaggle_spot_df[metric]>upperbound)),:]
    outliers_df
    ### maybe i should look at popularity beyond an upper bound, instead of more than average popularity
    popular_outliers_df=outliers_df.loc[outliers_df['popularity']>avg_poularity]
    return popular_outliers_df


def lower_outlier_songs(metric,year):
    # finds all the quartile related stats for each metric in a given year
    annual_q_stats_df=kaggle_spot_df.loc[kaggle_spot_df['year']==year,spot_metric_names].describe()
    #find outliers in the acousticness

    iqr=annual_q_stats_df[metric]['75%']-annual_q_stats_df[metric]['25%']
    iqr
    upperbound=annual_q_stats_df[metric]['75%']+(1.5*iqr)
    lowerbound=annual_q_stats_df[metric]['25%']-(1.5*iqr)

    # i could look at above average popularity or popularity above the 75 percentile
    avg_poularity=annual_q_stats_df['popularity']['mean']
    high_popularity=annual_q_stats_df['popularity']['75%']


    outliers_df=kaggle_spot_df.loc[(kaggle_spot_df['year']==year)&((kaggle_spot_df[metric]<lowerbound)),:]
    ### maybe i should look at popularity beyond an upper bound, instead of more than average popularity
    popular_outliers_df=outliers_df.loc[outliers_df['popularity']>avg_poularity]
    return popular_outliers


# outlier_songs('acousticness',1928)


# upper_outlier_songs('energy',2010)
# lower_outlier_songs('energy',2010)


In [42]:
## loop through some years and find the most danceable songs

In [43]:
# finds the most danceable song for each year, creating a dataframe

years_list=kaggle_spot_df['year'].unique().tolist()

dance_playlist_df=pd.DataFrame()


for year in years_list:
    songs_per_year_df = kaggle_spot_df.loc[kaggle_spot_df['year']==year,:]
    annual_dance_songs_df=songs_per_year_df.loc[songs_per_year_df['danceability']==songs_per_year_df['danceability'].max(),:]
    dance_playlist_df=dance_playlist_df.append(annual_dance_songs_df)



print(len(dance_playlist_df))
print(len(years_list))
dance_playlist_df.head(20)


103
100


,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
116950,0.986,['Francisco Canaro'],0.901,173107,0.201,0,1ymwnvDvUNZcEaN8tx8o2f,0.001480,4,0.0843,-9.719,1,Bulincito de Estudiante - Instrumental (Remast...,0,1928-09-16,0.1230,113.562,0.903,1928
55783,0.976,"['Francisco Canaro', 'Ernesto Fama']",0.901,186987,0.480,0,7vNGVLvicV4mWSeCxTdW19,0.143000,7,0.0597,-6.153,1,El Gringo Bonachón - Remasterizado,0,1930-12-05,0.1200,114.753,0.934,1930
161869,0.592,['Avelino Muñoz'],0.921,167800,0.749,0,4Y7yZIB7SsMbkCjXsG7Ucq,0.046400,10,0.3230,-11.180,1,Cuchareta,0,1935-01-01,0.1830,118.057,0.963,1935
117720,0.995,['Alvarenga E Ranchinho'],0.849,185667,0.157,0,141nPuknO62gDdA9X1sIjc,0.303000,2,0.1230,-15.615,1,A Moda dos Meses,1,1936-01-01,0.1480,92.870,0.662,1936
80025,0.895,['Melody the Music Box'],0.917,106826,0.298,0,3Lpl0D7VkHcJSVwxgYz9wu,0.923000,7,0.1050,-5.569,1,I've Got No Strings,3,1940,0.0886,130.105,0.838,1940
24606,0.225,['Orchestra Studio 7'],0.940,191739,0.435,0,6dMYpcrSobVXuDReQTiGS3,0.000669,9,0.0366,-10.849,1,Giuvanne cu 'a chitarre - Instrument and base ...,0,1942-11-13,0.2940,113.891,0.704,1942
55963,0.905,['Chapuseaux y Damirón'],0.894,191869,0.380,0,5OuubbWQGTQznRoaTeMlaG,0.003780,7,0.0592,-16.039,1,Se Acabó el Carbón,0,1945-01-01,0.0566,117.843,0.949,1945
154069,0.989,['S.N.Tripathi'],0.916,191048,0.315,0,3wIxTyp6zLGn61A6rHvIUN,0.000001,0,0.1900,-7.308,1,Dupatta Mora Aaj Hi Rang Dijo,0,1946-12-31,0.2200,120.118,0.728,1946
162655,0.992,['Zeenat Begum'],0.877,207393,0.210,0,4jio5c8grHcWMCUDf45rHC,0.168000,2,0.2700,-10.313,0,Gaa Meri Maina Gaa,0,1947-12-31,0.2510,123.078,0.666,1947
56067,0.995,"['Geeta Dutt', 'Nalini Jaywant']",0.900,184985,0.267,0,7fkCkq9mpsCkH9ztFaomQT,0.816000,0,0.1870,-7.240,1,Aavyo Hun Bangadiwalo,0,1948-09-20,0.3790,120.456,0.694,1948


In [44]:
### trying to create a playlist in spotify for the dance_playlist_df
### something is wrong with this code, i'm geting a 403 error


#sets up authorization and gets token
AUTH_URL = 'https://accounts.spotify.com/api/token'

auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': api_keys.spot_client_id,
    'client_secret': api_keys.spot_client_secret,
})

auth_response_data = auth_response.json()
access_token = auth_response_data['access_token']

#sets the headers for the request
playlist_creation_headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token),
    "Content-Type":'application/json',
    
}
#sets the body parameters for the request
playlist_creation_body_param={
    'name':"api gen playlist",
    'public':True,
    'description':"playlist generated via api"

}


#sets the base url and user playlist url



userplaylists_url= f'https://api.spotify.com/v1/users/{api_keys.spot_user_id}/playlists'


# requests.get(userplaylists_url,headers=playlist_creation_headers,params=playlist_creation_body_param)

# response=requests.get(userplaylists_url,headers=playlist_creation_headers,params=playlist_creation_body_param)


# #spits out the playlists on my account
# response.json()

#trying post instead of get 

pushy_response=requests.post(userplaylists_url,headers=playlist_creation_headers,params=playlist_creation_body_param)




print(access_token)
print(type(access_token))
pushy_response

### useful links regarding creating a playlist and adding songs to a play list using spotify's api:
# https://developer.spotify.com/documentation/web-api/reference/playlists/create-playlist/
# https://developer.spotify.com/documentation/web-api/reference/playlists/add-tracks-to-playlist/




##### also, i'm wondering if i could/should use all the song id's to request the # of plays for each song from spotify's API
#### also, I'm wondering if we could weight each year(or era) by the average popularity of a song within that year(or era), and use it to compare spotify metrics accross years(or eras)

BQAJbZC-D3AuecUuT6zitZM-4NHl5VFQNg5_2_MTXr958T3nBXHeuRj0Eg7g9o-APrhfnXVds1rHVReChAA
<class 'str'>


<Response [403]>